In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
raw_mail_data = pd.read_csv(r'D:\Harshith\spam/mail_data.csv')
raw_mail_data 

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
null_values = raw_mail_data.isnull().sum()
null_values

Category    0
Message     0
dtype: int64

In [4]:
raw_mail_data['Category'] = raw_mail_data['Category'].replace({
    'spam': 0,
    'ham': 1
})

In [5]:
raw_mail_data

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


In [6]:
X = raw_mail_data['Message']
X
Y = raw_mail_data['Category']
Y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: int64

In [7]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [9]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase='True')
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)
# Y_train = Y_train.astype('int')
# Y_test = Y_test.astype('int')

In [10]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression()

# Perform 5-fold cross-validation
scores = cross_val_score(model, X_train_features, Y_train, cv=10)

print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())


Cross-validation scores: [0.95964126 0.94170404 0.9529148  0.93273543 0.9529148  0.94170404
 0.94618834 0.95505618 0.95280899 0.93932584]
Average cross-validation score: 0.9474993701818913


In [11]:
model.fit(X_train_features, Y_train)

LogisticRegression()

In [12]:
prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)
accuracy_on_training_data

0.9670181736594121

In [13]:
prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)
accuracy_on_test_data

0.9659192825112107

In [14]:
#Example Case
input_mail = ["Dear user, You have been selected as the lucky winner of a $1,000 gift card. Claim your prize now by clicking on the link below. Hurry, this offer expires soon!Click here to claim your reward: [malicious-link]"]

input_data_features = feature_extraction.transform(input_mail)
prediction = model.predict(input_data_features)
print(prediction)
if (prediction[0]==1):
  print('Ham mail')

else:
  print('Spam mail')

[0]
Spam mail


In [16]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('feature_extraction.pkl', 'wb') as f:
    pickle.dump(feature_extraction, f)

In [17]:
%gui tk

In [ ]:
import tkinter as tk
from tkinter import messagebox

import numpy as np

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('feature_extraction.pkl', 'rb') as f:
    feature_extraction = pickle.load(f)

# Define the predict function
def predict_text():
    # Get input text from the GUI
    input_text = text_entry.get("1.0", "end-1c")

    # Preprocess the text and predict
    input_features = feature_extraction.transform([input_text])
    prediction = model.predict(input_features)[0]
    prediction_label = "Spam" if prediction == 1 else "Not Spam"

    # Show the prediction result
    messagebox.showinfo("Prediction", f"The text is classified as: {prediction_label}")

# Set up the GUI window
window = tk.Tk()
window.title("Spam Detection")

# Create a text input field and label
tk.Label(window, text="Enter text:").pack(pady=10)
text_entry = tk.Text(window, height=10, width=50)
text_entry.pack(pady=10)

# Create a button to trigger the prediction
predict_button = tk.Button(window, text="Predict", command=predict_text)
predict_button.pack(pady=10)

# Run the GUI
window.mainloop()
